In [ ]:
import pandas as pd
import numpy as np

"""
Data Loading
"""

# Read in data into a dataframe 
df = pd.read_csv('data/ny_energy_2016.csv')

df.shape

In [ ]:
# Print the information of the first instance
df.info()

In [ ]:
# Converts columns that should be numeric to float 

for col in list(df.columns):
    # Select columns that should be numeric
    if('ft²' in col or 
        'kBtu' in col or
        'Metric Tons CO2e' in col or
        'kWh' in col or
        'therms' in col or
        'gal' in col or
        'Score' in col):
            
            # Convert the data type to float
            df[col] = df[col].astype(float)


df.info()

In [ ]:
"""
Missing Values
"""

# Getting a sense of how many missing values are in each column
ser_miss_cnt = df.isnull().sum().sort_values(ascending = False)
ser_miss_rate = ser_miss_cnt / len(df)
print(ser_miss_cnt)
print(ser_miss_rate)

In [ ]:
# Remove any columns with more than threshold of missing rate
threshold = 0.5

print(ser_miss_rate[ser_miss_rate > 0.5])
del_list = ser_miss_rate[ser_miss_rate > 0.5].index

In [ ]:
# Drop the columns
df.drop(columns=del_list, inplace=True)
df.head()

In [ ]:
"""
Exploratory Data Analysis
"""

# Visualization setup
%matplotlib
%config InlineBackend.figure_format = 'svg'

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

import seaborn as sns
sns.set()  # set plot styles

In [ ]:
# Rename the score
df = df.rename(columns = {'ENERGY STAR Score': 'Score'})

# Plot the histogram of the Energy Star Score
plt.style.use('fivethirtyeight')
plt.hist(df['Score'].dropna(), bins = 100, edgecolor = 'k')
plt.xlabel('Score')
plt.ylabel('Number of Buildings')
plt.title('Energy Star Score Distribution')

In [ ]:
# Rename the score
df = df.rename(columns = {'ENERGY STAR Score': 'Score'})

# Plot the histogram of the Energy Star Score
plt.style.use('fivethirtyeight')
plt.hist(df['Score'].dropna(), bins = 100, edgecolor = 'k')
plt.xlabel('Score')
plt.ylabel('Number of Buildings')
plt.title('Energy Star Score Distribution')

In [ ]:
# Plot the histogram of Site EUI
plt.hist(df['Site EUI (kBtu/ft²)'].dropna(), bins = 20, edgecolor = 'black')
plt.xlabel('Site EUI')
plt.ylabel('Count')
plt.title('Site EUI Distribution')

In [ ]:
# Descriptive statistics
df['Site EUI (kBtu/ft²)'].describe()

In [ ]:
df['Site EUI (kBtu/ft²)'].dropna().sort_values().tail(10)

In [ ]:
df.loc[df['Site EUI (kBtu/ft²)'] == 869265, :]

In [ ]:
"""
Removing Outliers
- IQR-based outlier detection and elimination
- Definition of an extreme outlier:
    x < Q1 - 3IQR or x > Q3 - 3IQR
"""

# Calculate Q1 and Q3
q1 = df['Site EUI (kBtu/ft²)'].quantile(0.25)
q3 = df['Site EUI (kBtu/ft²)'].quantile(0.75)

print('Q1 =', q1)
print('Q3 =', q3)

In [ ]:
# IQR
iqr = q3-q1

# Upper/lower fences
# 1.5: mild outliers
# 3: extreme outliers

upper_fence = q3 + 3*iqr
lower_fence = q1 - 3*iqr

print('IQR =', iqr)
print('Upper fence =', upper_fence)
print('Lower fence =', lower_fence)

In [ ]:
# Obtain a refined dataset (without extreme outliers)
df = df[ (df['Site EUI (kBtu/ft²)'] < upper_fence) &
      (df['Site EUI (kBtu/ft²)'] > lower_fence) ]

df['Site EUI (kBtu/ft²)'].shape

In [ ]:
# Histogram of Site EUI after outlier eliminations

plt.hist(df['Site EUI (kBtu/ft²)'].dropna(), bins = 20, edgecolor = 'black')
plt.xlabel('Site EUI')
plt.ylabel('Count')
plt.title('Site EUI Distribution')

In [ ]:
"""
Looking for Relationships
- Investigate categorical variables and the target variable (Energy Star Score)
- Categorical variable 1: Largest Property Use Type: Property type of a building
- Categorical variable 2: Borough
"""

# Relationships between property types and scores

df_has_score = df.dropna(subset=['Score'])  # Select data with Score
types = df_has_score['Largest Property Use Type'].value_counts()   # Count buildings by the property type
types

In [ ]:
# Select property types whose frequencies are more than 100
types_100 = types[types.values > 100].index
types_100

In [ ]:
# Plot the distribution of Scores for property types

for b_type in types_100:
    # Select buildings by property type
    subset = df[df['Largest Property Use Type'] == b_type]
    
    # Density plot of Energy Star Scores
    sns.kdeplot(subset['Score'].dropna(),
               label=b_type, alpha=0.8)

plt.xlabel('Energy Star Scores')
plt.ylabel('Density')
plt.title('Density Plot of Energy Star Scores by Building Type')
plt.legend(loc='best')

In [ ]:
# Examine another categorical variable 'Borough'
boroughs = df_has_score['Borough'].value_counts()  # Count buildings by the borough
print(boroughs)

boroughs = boroughs.index
boroughs

In [ ]:
# Plot each borough distribution of scores
for borough in boroughs:
    # Select buildings by borough
    subset = df[df['Borough'] == borough]
    
    # Density plot of Energy Star Scores
    sns.kdeplot(subset['Score'].dropna(),
               label=borough)

plt.xlabel("Energy Star Scores")
plt.ylabel('Density')
plt.title("Density Plot of Energy Star Scores by Borough")
plt.legend(loc='best')

In [ ]:
"""
Correlations between Features and Target Variable
"""

# Calculate Pearson correlation coefficients between each feature and the target variable
corr = df.corr()['Score'].sort_values()
corr

In [ ]:
"""
Two-Variable Plots
- Variable 1: Site EUI
- Variable 2: Largest Property Use Type
- Target: Energy Star Score
"""

# Extract the building types
df['Largest Property Use Type'] = df.dropna(subset = ['Score'])['Largest Property Use Type']

In [ ]:
# Limit to building types with more than 100 observations
print(types_100)
df = df[df['Largest Property Use Type'].isin(types_100)]
df.shape

In [ ]:
# Scatterplot of Score vs Site EUI

sns.lmplot('Site EUI (kBtu/ft²)', 'Score',
          hue='Largest Property Use Type', data=df,
          scatter_kws={'alpha': 0.8}, fit_reg=False,
          size=12, aspect=1.2)

plt.xlabel('Site EUI')
plt.ylabel('Enery Star Score')
plt.title('Energy Star Score vs Site EUI')

In [ ]:
"""
Pairs Plot
"""

# Select the columns to plot
df_plot = df[['Score',
              'Site EUI (kBtu/ft²)',
              'Weather Normalized Site EUI (kBtu/ft²)',
              'Weather Normalized Source EUI (kBtu/ft²)',
              'Source EUI (kBtu/ft²)',
              'Weather Normalized Site Electricity Intensity (kWh/ft²)',
              'Total GHG Emissions (Metric Tons CO2e)']]

In [ ]:
# Rename columns
df_plot.rename(columns={'Site EUI (kBtu/ft²)':
                        'Site EUI',
                       'Weather Normalized Site EUI (kBtu/ft²)':
                        'Weather Norm Site EUI',
                       'Weather Normalized Source EUI (kBtu/ft²)':
                        'Weather Norm Src EUI)',
                       'Source EUI (kBtu/ft²)':
                        'Source EUI',
                       'Weather Normalized Site Electricity Intensity (kWh/ft²)':
                        'Weather Norm Site Elec Intensity',
                       'Total GHG Emissions (Metric Tons CO2e)':
                       'Total GHG Emissions'}, inplace=True)